# Clinical Trials Landscape Analysis

**Objective:**  
This project builds an end-to-end ETL pipeline using the ClinicalTrials.gov Data API. 
Clinical trial data is extracted via API calls, transformed and normalized using Python 
(pandas), loaded into a SQL database, and analyzed to uncover trends in trial activity, 
sponsor behavior, and completion outcomes.

**Key Questions:**
- How many stroke trials are completed vs ongoing?
- Where are stroke trials geographically concentrated?
- What types of interventions dominate stroke research?
- How large are stroke trials (enrollment)?
- What outcomes are most commonly measured?


## Data Source

- **API:** ClinicalTrials.gov Data API (v2)
- **Base URL:** https://clinicaltrials.gov/api/v2
- **Endpoint Used:** /studies
- **Authentication:** None (public API)
- **Response Format:** JSON

The ClinicalTrials.gov database is maintained by the U.S. National Library of Medicine 
and contains information on privately and publicly funded clinical studies conducted 
around the world.


## Project Architecture

1. Extract clinical trial data from the ClinicalTrials.gov API
2. Parse nested JSON fields into Pandas DataFrame
3. Transform and standardize clinical trial data using pandas (deduplication, type enforcement, and normalization)
4. Load cleaned data into a relational SQL database
5. Perform SQL-based analysis to answer research questions


In [1]:
#Import libraries 
import os 
import json
import requests 
import pandas as pd 
import numpy as np
import sqlite3
from dotenv import load_dotenv 
from datetime import datetime
import time

In [2]:
#url for the API
url = "https://clinicaltrials.gov/api/v2/studies"

PAGE_SIZE = 100               # records per API call
MAX_PAGES = 10                # limit for development/testing

QUERY_TERM = "stroke"               # condition of interest
STATUS_FILTER = "Completed"   # study completed or not 

In [3]:
#parameters for the API call 
params = {"query.term": QUERY_TERM,
    "pageSize": PAGE_SIZE,
    }

EXTRACT 

In [4]:
#response and response code 
response = requests.get(url, params)
response.status_code

200

In [5]:
data = response.json()

Extract Data for Studies Table 



In [6]:
#create column names and rows for DataFrame 
rows_studies = []
column_names_studies = [
    "nct_id",
    "brief_title",
    "official_title",
    "acronym",
    "org_name",
    "org_class",
    "overall_status",
    "start_date",
    "primary_completion_date",
    "completion_date",
    "study_first_post_date",
    "last_update_post_date",
    "study_type",
    "allocation",
    "intervention_model",
    "primary_purpose",
    "masking",
    "enrollment_count",
    "has_dmc",
    "fda_regulated_drug",
    "fda_regulated_device"
]


#loop through each study in the studies list and extract the necessary column values 
for study in data['studies']:
    
    # index into the json creating module variables, ensuring {} fallbacks at each level
    protocol                = study.get('protocolSection', {})
    id_module               = protocol.get('identificationModule', {})
    status_module           = protocol.get('statusModule', {})
    design_module           = protocol.get('designModule', {})
    oversight_module        = protocol.get('oversightModule', {})
    
    #assign values, ensuring {} fallbacks at each level to handle missing values (NULL)
    nct_id                  = id_module.get('nctId')
    brief_title             = id_module.get('briefTitle')
    official_title          = id_module.get('officialTitle')
    acronym                 = id_module.get('acronym')
    org_name                = id_module.get('organization', {}).get('fullName')
    org_class               = id_module.get('organization', {}).get('class')
    overall_status          = status_module.get('overallStatus')
    start_date              = status_module.get('startDateStruct', {}).get('date')
    primary_completion_date = status_module.get('primaryCompletionDateStruct', {}).get('date')
    completion_date         = status_module.get('completionDateStruct', {}).get('date')
    study_first_post_date   = status_module.get('studyFirstPostDateStruct', {}).get('date')
    last_update_post_date   = status_module.get('lastUpdatePostDateStruct', {}).get('date')
    study_type              = design_module.get('studyType')
    allocation              = design_module.get('designInfo', {}).get('allocation')
    intervention_model      = design_module.get('designInfo', {}).get('interventionModel')
    primary_purpose         = design_module.get('designInfo', {}).get('primaryPurpose')
    masking                 = design_module.get('designInfo', {}).get('maskingInfo', {}).get('masking')
    enrollment_count        = design_module.get('enrollmentInfo', {}).get('count')
    has_dmc                 = oversight_module.get('oversightHasDmc')
    fda_regulated_drug      = oversight_module.get('isFdaRegulatedDrug')
    fda_regulated_device    = oversight_module.get('isFdaRegulatedDevice')
    
    #initialize a tuple representing the active row for each iteration 
    active_row = (nct_id, brief_title, official_title, acronym, org_name, org_class, overall_status, start_date, 
                  primary_completion_date, completion_date, study_first_post_date, last_update_post_date, study_type,
                  allocation, intervention_model, primary_purpose, masking, enrollment_count, has_dmc, fda_regulated_drug, 
                  fda_regulated_device)    
    #append the active row to the rows list 
    rows_studies.append(active_row)
    
    


In [7]:
df_studies = pd.DataFrame(rows_studies, columns=column_names_studies)

In [8]:
df_studies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   nct_id                   100 non-null    object
 1   brief_title              100 non-null    object
 2   official_title           99 non-null     object
 3   acronym                  46 non-null     object
 4   org_name                 100 non-null    object
 5   org_class                100 non-null    object
 6   overall_status           100 non-null    object
 7   start_date               99 non-null     object
 8   primary_completion_date  94 non-null     object
 9   completion_date          100 non-null    object
 10  study_first_post_date    100 non-null    object
 11  last_update_post_date    100 non-null    object
 12  study_type               100 non-null    object
 13  allocation               66 non-null     object
 14  intervention_model       65 non-null     ob

In [9]:
df_studies.head()

,nct_id,brief_title,official_title,acronym,org_name,org_class,overall_status,start_date,primary_completion_date,completion_date,...,last_update_post_date,study_type,allocation,intervention_model,primary_purpose,masking,enrollment_count,has_dmc,fda_regulated_drug,fda_regulated_device
0,NCT06974461,Effects of Blood Flow Restriction Training on ...,Effects of Blood Flow Restriction Training on ...,None,The Hong Kong Polytechnic University,OTHER,NOT_YET_RECRUITING,2025-05-15,2027-12-28,2027-12-28,...,2025-05-16,INTERVENTIONAL,RANDOMIZED,PARALLEL,TREATMENT,DOUBLE,64,True,False,False
1,NCT06365086,Targeting Personalized Brain States Reflecting...,Targeting Personalized Brain States Reflecting...,None,University of Texas at Austin,OTHER,COMPLETED,2023-01-27,2025-01-01,2025-01-01,...,2025-04-17,INTERVENTIONAL,NA,CROSSOVER,BASIC_SCIENCE,DOUBLE,21,False,False,True
2,NCT05574686,Telemonitoring Hypertension and Breast Cancer,Tracking Social Outcomes of Hypertension in Wo...,None,University of Mississippi Medical Center,OTHER,UNKNOWN,2023-02-01,2023-05-31,2023-05-31,...,2023-05-10,INTERVENTIONAL,NA,SINGLE_GROUP,SUPPORTIVE_CARE,NONE,40,True,False,False
3,NCT04795505,Effect of Tertiary A-level Hospital Remote Int...,Effect of Tertiary A-level Hospital Remote Int...,CHIPCHAT,"Chinese Academy of Medical Sciences, Fuwai Hos...",OTHER,COMPLETED,2018-09-01,2023-01-01,2023-01-01,...,2023-02-16,OBSERVATIONAL,None,None,None,None,1424,True,False,False
4,NCT04274205,Influence of Brief Supportive Psychotherapy on...,Influence of Brief Supportive Psychotherapy on...,Supportive,Indonesia University,OTHER,COMPLETED,2019-04-18,2019-07-30,2019-07-30,...,2020-02-20,INTERVENTIONAL,RANDOMIZED,PARALLEL,TREATMENT,NONE,72,None,False,False


Extract Data for Study Conditions Table 

In [10]:
rows_study_conditions = []

column_names_study_conditions = ['nct_id',
                                 'condition'
    
]

for study in data['studies']:
    
    protocol                = study.get('protocolSection', {})
    
    nct_id                  = protocol.get('identificationModule', {}).get('nctId')
    condition               = protocol.get('conditionsModule', {}).get('conditions', [])
    
    #iterate through the list of conditions and for each iteration append the condition with the corresponding nct_id
    for condition in condition:
        rows_study_conditions.append(
            {'nct_id' : nct_id,
             'condition' : condition}
        )
    
    

In [11]:
df_study_conditions = pd.DataFrame(rows_study_conditions, columns=column_names_study_conditions)

In [12]:
df_study_conditions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nct_id     235 non-null    object
 1   condition  235 non-null    object
dtypes: object(2)
memory usage: 3.8+ KB


In [13]:
df_study_conditions.head()

,nct_id,condition
0,NCT06974461,Sarcopenia
1,NCT06974461,Chronic Stroke Patients
2,NCT06365086,Neurotypical Adults
3,NCT05574686,Hypertension
4,NCT05574686,Breast Cancer


Extract Data for Study Interventions Table 

In [14]:
rows_study_interventions = []
column_names_study_interventions = ['nct_id',
                           'intervention_name',
                           'intervention_type',
]

for study in data['studies']:
    protocol                = study.get('protocolSection', {})
    interventions           = protocol.get('armsInterventionsModule', {}).get('interventions', [])
    
    
    nct_id                  = protocol.get('identificationModule', {}).get('nctId')
    
    #iterate through each intervention in the interventions list and append their name & type 
    #with their corresponding nct_id to the rows list 
    for intervention in interventions:
        rows_study_interventions.append({
            'nct_id' : nct_id,
            'intervention_name' : intervention.get('name'),
            'intervention_type' : intervention.get('type')
        })
    
    

In [15]:
df_study_interventions = pd.DataFrame(rows_study_interventions, columns = column_names_study_interventions)

In [16]:
df_study_interventions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   nct_id             163 non-null    object
 1   intervention_name  163 non-null    object
 2   intervention_type  163 non-null    object
dtypes: object(3)
memory usage: 3.9+ KB


In [17]:
df_study_interventions.head()

,nct_id,intervention_name,intervention_type
0,NCT06974461,Blood Flow Restriction (BFR) Walking,PROCEDURE
1,NCT06974461,Sham Blood Flow Restriction (Sham-BFR) Walking,PROCEDURE
2,NCT06365086,Personalized brain state-dependent single-puls...,DEVICE
3,NCT05574686,Telehealth Blood Pressure Monitoring and Food ...,BEHAVIORAL
4,NCT04795505,Intervention,BEHAVIORAL


Extract Data for Primary Outcomes Table

In [18]:
rows_primary_outcomes = []

column_names_primary_outcomes = [
    'nct_id',
    'outcome_measure',
    'time_frame',
]

for study in data['studies']:
    protocol                = study.get('protocolSection', {})
    primary_outcomes        = protocol.get('outcomesModule', {}).get('primaryOutcomes', [])
    
    nct_id                  = protocol.get('identificationModule', {}).get('nctId')
    
    #iterate through each primary outcome and append the measure & time frame with the corresponding nct_id
    #to the rows list
    for outcome in primary_outcomes:
        rows_primary_outcomes.append({
            'nct_id' : nct_id,
            'outcome_measure' : outcome.get('measure'),
            'time_frame' : outcome.get('timeFrame')
        })

In [19]:
df_primary_outcomes = pd.DataFrame(rows_primary_outcomes, columns=column_names_primary_outcomes)

In [20]:
df_primary_outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nct_id           274 non-null    object
 1   outcome_measure  274 non-null    object
 2   time_frame       270 non-null    object
dtypes: object(3)
memory usage: 6.6+ KB


In [21]:
df_primary_outcomes.head()

,nct_id,outcome_measure,time_frame
0,NCT06974461,Muscle Strength,"baseline (prior to intervention), 6 weeks post..."
1,NCT06974461,Motor Function - Fugl-Meyer Assessment (FMA),"baseline (prior to intervention), 6 weeks post..."
2,NCT06974461,Aerobic Capacity and Endurance - Six-Minute Wa...,"baseline (prior to intervention), 6 weeks post..."
3,NCT06974461,Mobility - Timed Up and Go Test (TUG),"baseline (prior to intervention), 6 weeks post..."
4,NCT06974461,Gait Speed - 10-Meter Walk Test (10MWT),"baseline (prior to intervention), 6 weeks post..."


Extract Data for Secondary Outcomes Table 

In [22]:
rows_secondary_outcomes = []

column_names_secondary_outcomes = [
    'nct_id',
    'outcome_measure',
    'time_frame'
]


for study in data['studies']:
    protocol                = study.get('protocolSection', {})
    nct_id                  = protocol.get('identificationModule', {}).get('nctId')
    
    secondary_outcomes      = protocol.get('outcomesModule', {}).get('secondaryOutcomes', [])
    
    #iterate through each secondary outcome and append the measure & time frame with the corresponding nct_id
    #to the rows list
    for outcome in secondary_outcomes:
        rows_secondary_outcomes.append({
            'nct_id' : nct_id,
            'outcome_measure' : outcome.get('measure'),
            'time_frame' : outcome.get('timeFrame')
        })
    

In [23]:
df_secondary_outcomes = pd.DataFrame(rows_secondary_outcomes, columns=column_names_secondary_outcomes)

In [24]:
df_secondary_outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532 entries, 0 to 531
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nct_id           532 non-null    object
 1   outcome_measure  532 non-null    object
 2   time_frame       522 non-null    object
dtypes: object(3)
memory usage: 12.6+ KB


In [25]:
df_secondary_outcomes.head()

,nct_id,outcome_measure,time_frame
0,NCT06974461,Nutritional Status - Mini Nutritional Assessme...,"baseline (prior to intervention), 6 weeks post..."
1,NCT06974461,Quality of Life - Sarcopenia and Quality of Li...,"baseline (prior to intervention), 6 weeks post..."
2,NCT06974461,Appendicular Skeletal Muscle Mass (ASM),"Baseline, Week 6, Week 12"
3,NCT06974461,Muscle Thickness (MT),"Baseline, Week 6, Week 12"
4,NCT06974461,Pennation Angle,"Baseline, Week 6, Week 12"


Extract Data for Eligibility Summary Table 

In [26]:
rows_eligibility_summary = []

column_names_eligibility_summary = [
    'nct_id',
    'sex',
    'min_age',
    'max_age',
    'healthy_volunteers'
]

for study in data['studies']:
    protocol                = study.get('protocolSection', {})
    eligibility_module      = protocol.get('eligibilityModule', {})
    
    nct_id                  = protocol.get('identificationModule', {}).get('nctId')
    sex                     = eligibility_module.get('sex')
    min_age                 = eligibility_module.get('minimumAge')
    max_age                 = eligibility_module.get('maximumAge')
    healthy_volunteers      = eligibility_module.get('healthyVolunteers')
    
    active_row = (nct_id, sex, min_age, max_age, healthy_volunteers)
    
    rows_eligibility_summary.append(active_row)

In [27]:
df_eligibility_summary = pd.DataFrame(rows_eligibility_summary, columns=column_names_eligibility_summary)

In [28]:
df_eligibility_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   nct_id              100 non-null    object
 1   sex                 100 non-null    object
 2   min_age             94 non-null     object
 3   max_age             44 non-null     object
 4   healthy_volunteers  98 non-null     object
dtypes: object(5)
memory usage: 4.0+ KB


In [29]:
df_eligibility_summary.head()

,nct_id,sex,min_age,max_age,healthy_volunteers
0,NCT06974461,ALL,50 Years,None,False
1,NCT06365086,ALL,18 Years,None,True
2,NCT05574686,FEMALE,18 Years,None,False
3,NCT04795505,ALL,18 Years,99 Years,False
4,NCT04274205,ALL,18 Years,None,False


Extract Data for Study Locations Table 

In [30]:
rows_study_locations = []

column_names_study_locations = [
    'nct_id',
    'facility',
    'city',
    'country',
    'latitude',
    'longitude'
]

for study in data['studies']:
    protocol                = study.get('protocolSection', {})
    locations_module        = protocol.get('contactsLocationsModule', {})
    locations               = locations_module.get('locations', [])
    
    nct_id                  = protocol.get('identificationModule', {}).get('nctId')
    
    #iterate through each location and append the facility, city, country, latitude, and longitude with the 
    #corresponding nct_id to the rows list 
    for location in locations:
        facility            = location.get('facility')
        city                = location.get('city')
        country             = location.get('country')
        latitude            = location.get('geoPoint', {}).get('lat')
        longitude           = location.get('geoPoint', {}).get('lon')
        
        active_row = (nct_id, facility, city, country, latitude, longitude)
        
        rows_study_locations.append(active_row)

In [31]:
df_study_locations = pd.DataFrame(rows_study_locations, columns=column_names_study_locations)

In [32]:
df_study_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   nct_id     395 non-null    object 
 1   facility   395 non-null    object 
 2   city       395 non-null    object 
 3   country    395 non-null    object 
 4   latitude   388 non-null    float64
 5   longitude  388 non-null    float64
dtypes: float64(2), object(4)
memory usage: 18.6+ KB


In [33]:
df_study_locations.head()

,nct_id,facility,city,country,latitude,longitude
0,NCT06974461,Hong Kong Polytechnic University,Hung Hom,Hong Kong,22.30715,114.18532
1,NCT06365086,University of Texas at Austin,Austin,United States,30.26715,-97.74306
2,NCT05574686,University of Mississippi Medical Center Cance...,Jackson,United States,32.29876,-90.18481
3,NCT04795505,"Fuwai Hospital, Chinese Academy of Medical Sci...",Beijing,China,39.90750,116.39723
4,NCT04274205,Medical Faculty,Jakarta Pusat,Indonesia,-6.18180,106.82230


Extract Data for Study References Table 

In [34]:
rows_study_references = []

column_names_study_references = [
    'nct_id',
    'pmid',
    'citation'
]

for study in data['studies']:
    protocol                = study.get('protocolSection', {})
    references_module       = protocol.get('referencesModule', {})
    references              = references_module.get('references', [])
    
    nct_id                  = protocol.get('identificationModule', {}).get('nctId')
    
    #iterate through each references and append the pmid and citation with the corresponding nct_id to the rows list 
    for reference in references:
        pmid                = reference.get('pmid')
        citation            = reference.get('citation')
        
        active_row = (nct_id, pmid, citation)
        
        rows_study_references.append(active_row)

In [35]:
df_study_references = pd.DataFrame(rows_study_references, columns=column_names_study_references)

In [36]:
df_study_references.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   nct_id    276 non-null    object
 1   pmid      264 non-null    object
 2   citation  276 non-null    object
dtypes: object(3)
memory usage: 6.6+ KB


In [37]:
df_study_references.head()

,nct_id,pmid,citation
0,NCT06365086,39716573,"Khatri UU, Pulliam K, Manesiya M, Cortez MV, M..."
1,NCT05574686,33954415,"Mendy VL, Rowell-Cunsolo T, Bellerose M, Varga..."
2,NCT05574686,31298210,"Berkowitz SA, Basu S, Gundersen C, Seligman HK..."
3,NCT05574686,32185742,"Williams MS, Beech BM, Griffith DM, Jr Thorpe ..."
4,NCT04795505,30998400,"Zheng X, Spatz ES, Bai X, Huo X, Ding Q, Horak..."


Transform

Clean Studies Table 

Data Type Enforcement 

In [38]:
#enforce date date type for date columns, coercing errors to NaT
date_cols = [
    'start_date',
    'primary_completion_date',
    'completion_date',
    'study_first_post_date',
    'last_update_post_date'
]

for col in date_cols:
    df_studies[col] = pd.to_datetime(df_studies[col], errors='coerce')

In [39]:
#enforce boolean type for boolean columns
bool_cols = [
    'has_dmc',
    'fda_regulated_drug',
    'fda_regulated_device'  
]

for col in bool_cols:
    df_studies[col] = df_studies[col].astype(bool)

In [40]:
df_studies['enrollment_count'] = pd.to_numeric(df_studies['enrollment_count'], errors='coerce')

Deduplication & Keys

In [41]:
df_studies['nct_id'].is_unique

True

Business Logic Transformations

In [42]:
#add a new column for the duration of the study in days 
df_studies['duration_days'] = (df_studies['completion_date'] - df_studies['start_date']).dt.days

In [43]:
#add a new column for the enrollment size of the study in categories of (Small, Medium, Large, Very Large)
df_studies["enrollment_size"] = pd.cut(
    df_studies["enrollment_count"],
    bins=[0, 50, 200, 1000, float("inf")],
    labels=["Small", "Medium", "Large", "Very Large"]
)


In [44]:
status_map = {
    "COMPLETED": "Completed",
    "RECRUITING": "Ongoing",
    "ACTIVE_NOT_RECRUITING": "Ongoing",
    "ENROLLING_BY_INVITATION": "Ongoing",
    "TERMINATED": "Stopped",
    "WITHDRAWN": "Stopped",
    "SUSPENDED": "Stopped"
}

df_studies["overall_status"] = df_studies["overall_status"].map(status_map)

Clean Study Conditions Table

Remove Bad Duplicates (Dimension Table)

In [45]:
df_study_conditions = df_study_conditions.drop_duplicates(
    subset=["nct_id", "condition"]
)

Text Normalization

In [46]:
df_study_conditions['condition'] = df_study_conditions['condition'].str.strip()

Clean Study Interventions Table 

Remove Bad Duplicates (Dimension Table)

In [47]:
df_study_interventions = df_study_interventions.drop_duplicates(
    subset=['nct_id', 'intervention_name', 'intervention_type']
)

Text Normalization

In [48]:
df_study_interventions["intervention_name"] = (
    df_study_interventions["intervention_name"]
    .str.strip()
)

df_study_interventions["intervention_type"] = (
    df_study_interventions["intervention_type"]
    .str.upper()
)

Clean Primary Outcomes Table 

Text Normalization

In [49]:
df_primary_outcomes["outcome_measure"] = (
    df_primary_outcomes["outcome_measure"].str.strip()
)

Drop Bad Duplicates (Dimension Table)

In [50]:
df_primary_outcomes = df_primary_outcomes.drop_duplicates(
    subset=["nct_id", "outcome_measure", "time_frame"]
)

Clean Secondary Outcomes Table 

Text Normalization (Dimension Table)

In [51]:
df_secondary_outcomes['outcome_measure'] = (
    df_secondary_outcomes['outcome_measure'].str.strip()
)

Drop Bad Duplicates

In [52]:
df_secondary_outcomes = df_secondary_outcomes.drop_duplicates(
    subset=['nct_id', 'outcome_measure', 'time_frame']
)

Clean Eligibility Summary Table

Standardize Age Fields

In [53]:
#df_eligibility_summary['min_age'] = (df_eligibility_summary['min_age'].str.extract(r'(\d+)').astype(float))
#df_eligibility_summary['max_age'] = (df_eligibility_summary['max_age'].str.extract(r'(\d+)').astype(float))


Enforce Data Type (Boolean Type)

In [54]:
df_eligibility_summary['healthy_volunteers'] = df_eligibility_summary['healthy_volunteers'].astype('boolean')
df_eligibility_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   nct_id              100 non-null    object 
 1   sex                 100 non-null    object 
 2   min_age             94 non-null     object 
 3   max_age             44 non-null     object 
 4   healthy_volunteers  98 non-null     boolean
dtypes: boolean(1), object(4)
memory usage: 3.4+ KB


Clean Study Locations Table 

Text Normalization 

In [55]:
df_study_locations['facility'] = df_study_locations['facility'].str.strip()

df_study_locations['city'] = df_study_locations['city'].str.strip()

df_study_locations['country'] = df_study_locations['country'].str.strip()

df_study_locations

,nct_id,facility,city,country,latitude,longitude
0,NCT06974461,Hong Kong Polytechnic University,Hung Hom,Hong Kong,22.30715,114.18532
1,NCT06365086,University of Texas at Austin,Austin,United States,30.26715,-97.74306
2,NCT05574686,University of Mississippi Medical Center Cance...,Jackson,United States,32.29876,-90.18481
3,NCT04795505,"Fuwai Hospital, Chinese Academy of Medical Sci...",Beijing,China,39.90750,116.39723
4,NCT04274205,Medical Faculty,Jakarta Pusat,Indonesia,-6.18180,106.82230
...,...,...,...,...,...,...
390,NCT04034498,Ludwig Maximilians University,Munich,Germany,48.13743,11.57549
391,NCT04034498,Children Health Ireland,Dublin,Ireland,53.33306,-6.24889
392,NCT04034498,Queen Sirikit National Institute of Child Health,Bangkok,Thailand,13.75398,100.50144
393,NCT04034498,Prince of Songkla University,Hat Yai,Thailand,7.00836,100.47668


Remove Bad Duplicates 

In [56]:
df_study_locations = df_study_locations.drop_duplicates(
    subset=["nct_id", "facility", "city", "country"]
)

Lat & Long numeric type enforcement

In [57]:
df_study_locations.loc[:,'latitude'] = pd.to_numeric(df_study_locations['latitude'], errors='coerce') 

df_study_locations.loc[:,'longitude'] = pd.to_numeric(df_study_locations['longitude'], errors='coerce') 

Clean Study References Table

Remove Bad Duplicates

In [58]:
df_study_references = df_study_references.drop_duplicates(
    subset=["nct_id", "pmid"]
)



ALL TABLES CLEANED!!!

In [59]:
df_studies

,nct_id,brief_title,official_title,acronym,org_name,org_class,overall_status,start_date,primary_completion_date,completion_date,...,allocation,intervention_model,primary_purpose,masking,enrollment_count,has_dmc,fda_regulated_drug,fda_regulated_device,duration_days,enrollment_size
0,NCT06974461,Effects of Blood Flow Restriction Training on ...,Effects of Blood Flow Restriction Training on ...,None,The Hong Kong Polytechnic University,OTHER,NaN,2025-05-15,2027-12-28,2027-12-28,...,RANDOMIZED,PARALLEL,TREATMENT,DOUBLE,64,True,False,False,957.0,Medium
1,NCT06365086,Targeting Personalized Brain States Reflecting...,Targeting Personalized Brain States Reflecting...,None,University of Texas at Austin,OTHER,Completed,2023-01-27,2025-01-01,2025-01-01,...,NA,CROSSOVER,BASIC_SCIENCE,DOUBLE,21,False,False,True,705.0,Small
2,NCT05574686,Telemonitoring Hypertension and Breast Cancer,Tracking Social Outcomes of Hypertension in Wo...,None,University of Mississippi Medical Center,OTHER,NaN,2023-02-01,2023-05-31,2023-05-31,...,NA,SINGLE_GROUP,SUPPORTIVE_CARE,NONE,40,True,False,False,119.0,Small
3,NCT04795505,Effect of Tertiary A-level Hospital Remote Int...,Effect of Tertiary A-level Hospital Remote Int...,CHIPCHAT,"Chinese Academy of Medical Sciences, Fuwai Hos...",OTHER,Completed,2018-09-01,2023-01-01,2023-01-01,...,None,None,None,None,1424,True,False,False,1583.0,Very Large
4,NCT04274205,Influence of Brief Supportive Psychotherapy on...,Influence of Brief Supportive Psychotherapy on...,Supportive,Indonesia University,OTHER,Completed,2019-04-18,2019-07-30,2019-07-30,...,RANDOMIZED,PARALLEL,TREATMENT,NONE,72,False,False,False,103.0,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NCT04306471,Effect of Evolocumab in Patients With Critical...,Effect of Evolocumab in Patients With Critical...,Evol-CLI,University of Southern California,OTHER,NaN,2020-02-24,2021-02-04,2022-02-04,...,RANDOMIZED,PARALLEL,TREATMENT,TRIPLE,32,True,True,False,711.0,Small
96,NCT05276934,Brain Imaging After Non-traumatic Intracranial...,Prospective Assessment of Brain Imaging After ...,SAVEBRAINPWI,Erasme University Hospital,OTHER,Ongoing,2022-03-01,2025-10-20,2025-12-20,...,None,None,None,None,100,False,False,False,1390.0,Medium
97,NCT04122599,Melatonin for Prevention of Post Stroke Delirium,Melatonin for Prevention of Post Stroke Delirium,None,University Hospital Tuebingen,OTHER,NaN,2017-07-01,2019-11-01,2019-12-31,...,NA,SINGLE_GROUP,PREVENTION,NONE,2000,False,False,False,913.0,Very Large
98,NCT01359332,Evaluation of Therapeutic Hypothermia in Convu...,Evaluation of the Interests of a Therapeutic H...,HYBERNATUS,Assistance Publique - Hôpitaux de Paris,OTHER,Completed,NaT,NaT,NaT,...,RANDOMIZED,PARALLEL,TREATMENT,NONE,270,False,False,False,NaN,Large


In [60]:
df_study_conditions

,nct_id,condition
0,NCT06974461,Sarcopenia
1,NCT06974461,Chronic Stroke Patients
2,NCT06365086,Neurotypical Adults
3,NCT05574686,Hypertension
4,NCT05574686,Breast Cancer
...,...,...
230,NCT05276934,Intracranial Aneurysm
231,NCT05276934,Intracranial Hemorrhages
232,NCT04122599,Post Stroke Delirium
233,NCT01359332,Convulsive Status EPILEPTICUS


In [61]:
df_study_interventions

,nct_id,intervention_name,intervention_type
0,NCT06974461,Blood Flow Restriction (BFR) Walking,PROCEDURE
1,NCT06974461,Sham Blood Flow Restriction (Sham-BFR) Walking,PROCEDURE
2,NCT06365086,Personalized brain state-dependent single-puls...,DEVICE
3,NCT05574686,Telehealth Blood Pressure Monitoring and Food ...,BEHAVIORAL
4,NCT04795505,Intervention,BEHAVIORAL
...,...,...,...
158,NCT04306471,Evolocumab 140mg/mL Injector 1milliliter (mL) ...,DRUG
159,NCT04306471,Placebo 1 milliliter (mL) Injector Pen x 3 mon...,OTHER
160,NCT05276934,Brain computed tomography,DIAGNOSTIC_TEST
161,NCT04122599,Melatonin,DRUG


In [62]:
df_primary_outcomes

,nct_id,outcome_measure,time_frame
0,NCT06974461,Muscle Strength,"baseline (prior to intervention), 6 weeks post..."
1,NCT06974461,Motor Function - Fugl-Meyer Assessment (FMA),"baseline (prior to intervention), 6 weeks post..."
2,NCT06974461,Aerobic Capacity and Endurance - Six-Minute Wa...,"baseline (prior to intervention), 6 weeks post..."
3,NCT06974461,Mobility - Timed Up and Go Test (TUG),"baseline (prior to intervention), 6 weeks post..."
4,NCT06974461,Gait Speed - 10-Meter Walk Test (10MWT),"baseline (prior to intervention), 6 weeks post..."
...,...,...,...
269,NCT05276934,Change in brain hypoperfusion,"Change between day 0, 5, 9 after Intracranial ..."
270,NCT04122599,Change of delir prevalence,3months
271,NCT01359332,Functional impairment at 3 months,3 months
272,NCT04034498,To confirm the efficacy of Occlutech PmVSD in ...,1 year following implantation


In [63]:
df_secondary_outcomes

,nct_id,outcome_measure,time_frame
0,NCT06974461,Nutritional Status - Mini Nutritional Assessme...,"baseline (prior to intervention), 6 weeks post..."
1,NCT06974461,Quality of Life - Sarcopenia and Quality of Li...,"baseline (prior to intervention), 6 weeks post..."
2,NCT06974461,Appendicular Skeletal Muscle Mass (ASM),"Baseline, Week 6, Week 12"
3,NCT06974461,Muscle Thickness (MT),"Baseline, Week 6, Week 12"
4,NCT06974461,Pennation Angle,"Baseline, Week 6, Week 12"
...,...,...,...
527,NCT01359332,length of icu stay,3 months
528,NCT01359332,length of hospital stay,3 months
529,NCT01359332,Percentages of convulsive and non-convulsive s...,6 to 12 hours
530,NCT01359332,Seizure duration in minutes,3 days


In [64]:
df_eligibility_summary

,nct_id,sex,min_age,max_age,healthy_volunteers
0,NCT06974461,ALL,50 Years,None,False
1,NCT06365086,ALL,18 Years,None,True
2,NCT05574686,FEMALE,18 Years,None,False
3,NCT04795505,ALL,18 Years,99 Years,False
4,NCT04274205,ALL,18 Years,None,False
...,...,...,...,...,...
95,NCT04306471,ALL,40 Years,85 Years,False
96,NCT05276934,ALL,18 Years,100 Years,False
97,NCT04122599,ALL,55 Years,None,False
98,NCT01359332,ALL,18 Years,None,False


In [65]:
df_study_locations

,nct_id,facility,city,country,latitude,longitude
0,NCT06974461,Hong Kong Polytechnic University,Hung Hom,Hong Kong,22.30715,114.18532
1,NCT06365086,University of Texas at Austin,Austin,United States,30.26715,-97.74306
2,NCT05574686,University of Mississippi Medical Center Cance...,Jackson,United States,32.29876,-90.18481
3,NCT04795505,"Fuwai Hospital, Chinese Academy of Medical Sci...",Beijing,China,39.90750,116.39723
4,NCT04274205,Medical Faculty,Jakarta Pusat,Indonesia,-6.18180,106.82230
...,...,...,...,...,...,...
390,NCT04034498,Ludwig Maximilians University,Munich,Germany,48.13743,11.57549
391,NCT04034498,Children Health Ireland,Dublin,Ireland,53.33306,-6.24889
392,NCT04034498,Queen Sirikit National Institute of Child Health,Bangkok,Thailand,13.75398,100.50144
393,NCT04034498,Prince of Songkla University,Hat Yai,Thailand,7.00836,100.47668


In [66]:
df_study_references

,nct_id,pmid,citation
0,NCT06365086,39716573,"Khatri UU, Pulliam K, Manesiya M, Cortez MV, M..."
1,NCT05574686,33954415,"Mendy VL, Rowell-Cunsolo T, Bellerose M, Varga..."
2,NCT05574686,31298210,"Berkowitz SA, Basu S, Gundersen C, Seligman HK..."
3,NCT05574686,32185742,"Williams MS, Beech BM, Griffith DM, Jr Thorpe ..."
4,NCT04795505,30998400,"Zheng X, Spatz ES, Bai X, Huo X, Ding Q, Horak..."
...,...,...,...
271,NCT02032030,26418126,"Fritz BA, Kalarickal PL, Maybrier HR, Muench M..."
272,NCT01359332,37286846,"Jacq G, Crepon B, Resche-Rigon M, Schenck M, G..."
273,NCT01359332,32913029,"Fontaine C, Lemiale V, Resche-Rigon M, Schenck..."
274,NCT01359332,28002714,"Legriel S, Lemiale V, Schenck M, Chelly J, Lau..."


Upload to SQLite Database 

In [ ]:
import sqlite3
import pandas as pd 

#create connection to the SQLite database 
connection = sqlite3.connect('clinical_trials.db')

cursor = connection.cursor()

#create tables 
create_studies_table = """CREATE TABLE IF NOT EXISTS studies (
    nct_id TEXT PRIMARY KEY,
    brief_title TEXT, 
    official_title TEXT, 
    acronym TEXT, 
    org_name TEXT, 
    org_class TEXT, 
    overall_status TEXT, 
    start_date DATE,    
    primary_completion_date DATE,
    completion_date DATE,
    study_first_post_date DATE,
    last_update_post_date DATE,
    study_type TEXT,
    allocation TEXT,
    intervention_model TEXT,
    primary_purpose TEXT,
    masking TEXT,
    enrollment_count INTEGER,
    has_dmc BOOLEAN,
    fda_regulated_drug BOOLEAN,
    fda_regulated_device BOOLEAN,
    duration_days INTEGER,
    enrollment_size TEXT
    )"""
    
cursor.execute(create_studies_table)

#create study_conditions table 
create_study_conditions_table = """CREATE TABLE IF NOT EXISTS study_conditions (
    nct_id TEXT,
    condition TEXT,
    PRIMARY KEY (nct_id, condition),
    FOREIGN KEY (nct_id) REFERENCES studies(nct_id)
    )"""
    
cursor.execute(create_study_conditions_table)

#create study_interventions table
create_study_interventions_table = """CREATE TABLE IF NOT EXISTS study_interventions (
    nct_id TEXT,
    intervention_name TEXT,
    intervention_type TEXT,
    PRIMARY KEY (nct_id, intervention_name, intervention_type),
    FOREIGN KEY (nct_id) REFERENCES studies(nct_id)
    )"""
    
cursor.execute(create_study_interventions_table)

#create primary_outcomes table
create_primary_outcomes_table = """CREATE TABLE IF NOT EXISTS primary_outcomes (
    nct_id TEXT,
    outcome_measure TEXT,
    time_frame TEXT,
    PRIMARY KEY (nct_id, outcome_measure, time_frame),
    FOREIGN KEY (nct_id) REFERENCES studies(nct_id)
    )"""    
    
cursor.execute(create_primary_outcomes_table)   

#create secondary_outcomes table
create_secondary_outcomes_table = """CREATE TABLE IF NOT EXISTS secondary_outcomes (
    nct_id TEXT,
    outcome_measure TEXT,
    time_frame TEXT,
    PRIMARY KEY (nct_id, outcome_measure, time_frame),
    FOREIGN KEY (nct_id) REFERENCES studies(nct_id)
    )"""

cursor.execute(create_secondary_outcomes_table)

#create eligibility_summary table
create_eligibility_summary_table = """CREATE TABLE IF NOT EXISTS eligibility_summary (
    nct_id TEXT PRIMARY KEY,
    sex TEXT,
    min_age TEXT,   
    max_age TEXT,
    healthy_volunteers BOOLEAN,
    FOREIGN KEY (nct_id) REFERENCES studies(nct_id)
    )"""

cursor.execute(create_eligibility_summary_table)

#create study_locations table
create_study_locations_table = """CREATE TABLE IF NOT EXISTS study_locations (
    nct_id TEXT,
    facility TEXT,
    city TEXT,
    country TEXT,
    latitude REAL,
    longitude REAL,
    PRIMARY KEY (nct_id, facility, city, country),
    FOREIGN KEY (nct_id) REFERENCES studies(nct_id)
    )"""
    
cursor.execute(create_study_locations_table)

#create study_references table
create_study_references_table = """CREATE TABLE IF NOT EXISTS study_references (
    nct_id TEXT,
    pmid TEXT,
    citation TEXT,
    PRIMARY KEY (nct_id, pmid),
    FOREIGN KEY (nct_id) REFERENCES studies(nct_id)
    )"""
    
cursor.execute(create_study_references_table)

#commit the changes to the database
connection.commit()

In [71]:
#load DataFrames into their respective tables in the database 

df_studies.to_sql('studies', connection, if_exists='replace', index=False)

df_study_conditions.to_sql('study_conditions', connection, if_exists='replace', index=False)

df_study_interventions.to_sql('study_interventions', connection, if_exists='replace', index=False)

df_primary_outcomes.to_sql('primary_outcomes', connection, if_exists='replace', index=False)

df_secondary_outcomes.to_sql('secondary_outcomes', connection, if_exists='replace', index=False)

df_eligibility_summary.to_sql('eligibility_summary', connection, if_exists='replace', index=False)

df_study_locations.to_sql('study_locations', connection, if_exists='replace', index=False)

df_study_references.to_sql('study_references', connection, if_exists='replace', index=False)

connection.close()